**AIDAR**


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
# Import Drive API and authenticate.
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
all_files =['Historical_Token_Prices.csv', 'Net_inflow_and_aum.csv', ]

path = '/content/gdrive/My Drive/IndexCoop Capstone/CSV data files/'
my_pd = pd.DataFrame()

#First File Historical prices
for file in all_files:
  df = pd.DataFrame()
  df = pd.read_csv(path + file, index_col=None, parse_dates=True)
  df = df.rename(columns={"day": "Date"})
  df = df.reset_index(drop=True)
  df['Date'] = df['Date'].astype('datetime64[ns]')
  df.set_index('Date')
  my_pd = pd.concat([my_pd, df], axis=0) 

my_pd = my_pd[my_pd['symbol'] == 'DPI']
my_pd = my_pd.reset_index(drop=True)
my_pd.drop(columns=['symbol'])
my_pd = my_pd.rename(columns={"price": "DPI_price"})
my_pd.insert(2,'DPI_returns' ,my_pd['DPI_price'].pct_change())


In [ ]:
my_pd

,Date,symbol,DPI_returns,DPI_price,aum,gross_flow,net_flow
0,2020-09-10,DPI,NaN,456.753025,NaN,NaN,NaN
1,2020-09-11,DPI,-0.715199,130.083626,NaN,NaN,NaN
2,2020-09-12,DPI,0.021390,132.866086,NaN,NaN,NaN
3,2020-09-13,DPI,0.029888,136.837190,NaN,NaN,NaN
4,2020-09-14,DPI,-0.057124,129.020509,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1512,2022-10-06,DPI,0.000000,NaN,3.154795e+07,85911.082107,-85911.082107
1513,2022-10-07,DPI,0.000000,NaN,3.107695e+07,3820.074572,-3820.074572
1514,2022-10-08,DPI,0.000000,NaN,3.090662e+07,0.000000,0.000000
1515,2022-10-09,DPI,0.000000,NaN,3.043687e+07,85926.564808,-85926.564808


**NILAKSHI**

In [ ]:
import pandas as pd
import numpy as np
import os
os.getcwd() 
# Import Drive API and authenticate.
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
path = '/content/gdrive/My Drive/IndexCoop Capstone/DPI analysis/datasets'
underlying_assets = pd.read_csv(path + '/underl_assets_returns.csv', index_col ='date')
benchmarks = pd.read_csv(path + '/benchmarks_returns.csv', index_col ='date')
fear_index = pd.read_csv(path + '/crypto_fear_index_historical.csv', index_col ='timestamp')


ValueError: ignored

In [ ]:
path = '/content/gdrive/My Drive/IndexCoop Capstone/DPI analysis/datasets'
underlying_assets = pd.read_csv(path + '/underl_assets_returns.csv')
underlying_assets.rename(columns={'date': 'Date'}, inplace=True)
underlying_assets['Date'] = pd.to_datetime(underlying_assets['Date'])


In [ ]:
benchmarks = pd.read_csv(path + '/benchmarks_returns.csv')
benchmarks.rename(columns={'date': 'Date'}, inplace=True)
benchmarks['Date'] = pd.to_datetime(benchmarks['Date'])

In [ ]:
fear_index = pd.read_csv(path + '/crypto_fear_index_historical.csv')
fear_index.rename(columns={'timestamp': 'Date'}, inplace=True)
fear_index['Date'] = pd.to_datetime(fear_index['Date'])

In [ ]:
DPI_prices = pd.read_csv('/content/gdrive/My Drive/IndexCoop Capstone/CSV data files/Historical_Token_Prices.csv')
DPI_prices = DPI_prices.rename(columns={'day': 'Date'})
DPI_prices = DPI_prices.loc[DPI_prices['symbol'] == 'DPI'] 
DPI_prices['Date'] = DPI_prices['Date'].astype('datetime64[ns]') 
DPI_prices['DPI_returns'] = DPI_prices['price'].pct_change() 
DPI_prices = DPI_prices.drop(columns=['symbol', 'price'])



In [ ]:
DPI_prices

In [ ]:
AUM = pd.read_csv('/content/gdrive/My Drive/IndexCoop Capstone/CSV data files/Net_inflow_and_aum.csv')
AUM = AUM.loc[AUM['symbol'] == 'DPI'] 
AUM['aum_change'] = AUM['aum'].pct_change() 
AUM['net_flow_change'] = AUM['net_flow'].pct_change() 
AUM = AUM.drop(columns=['aum', 'symbol', 'gross_flow', 'net_flow'])
AUM = AUM.rename(columns={'day': 'Date'})
AUM['Date'] = AUM['Date'].astype('datetime64[ns]') 


In [ ]:
new_df = DPI_prices.merge(AUM, on ='Date')
new_df.set_index("Date",inplace=True)

In [ ]:
concat_df = new_df.merge(underlying_assets, on ="Date").merge(benchmarks, on='Date').merge(fear_index,on="Date")


In [ ]:
concat_df.set_index('Date',inplace=True)


In [ ]:
concat_df

In [ ]:
def fear_index_classification(data):
  data.loc[data['value_classification'] == 'Extreme Fear', 'fear_index'] = '-2'
  data.loc[data['value_classification'] == 'Fear', 'fear_index'] = '-1'
  data.loc[data['value_classification'] == 'Neutral', 'fear_index'] = '0'
  data.loc[data['value_classification'] == 'Greed', 'fear_index'] = '1'
  data.loc[data['value_classification'] == 'Extreme Greed', 'fear_index'] = '2'

In [ ]:
fear_index_classification(concat_df)


In [ ]:
concat_df.drop(columns=['value','value_classification'],inplace=True)

In [ ]:
correlations= concat_df.iloc[:,:-1].corr()
round(correlations,2) #Added (Adriana)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
#Just an idea for scatterplot to find colinearity
# sns.pairplot(concat_df);

plt.figure(figsize = (16,10))
sns.heatmap(correlations,cmap='crest', annot=True,)

In [ ]:
X= concat_df.iloc[:,1:-1]

In [ ]:
col = concat_df.pop('net_flow_change')
concat_df.insert(0,'net_flow_change',col)

In [ ]:
concat_df

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [ ]:
model_df = concat_df.copy()
model_df.replace([np.inf, -np.inf], np.nan, inplace=True)
model_df.dropna(inplace=True)


In [ ]:
X= model_df.iloc[:,1:-1]
Y= model_df['net_flow_change']

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [ ]:
model= LinearRegression().fit(X_train,Y_train)
Y_pred = model.predict(X_test)
#model.Summary() #Added (Adriana)

# New Section

In [ ]:
print(model.intercept_, model.score(X_train, Y_train))

In [ ]:
#print( model.coef_)

In [ ]:
r2_score(Y_test,Y_pred)

In [ ]:

##NEW DF

market_data = pd.read_csv('/content/gdrive/My Drive/IndexCoop Capstone/CSV data files/Market_Data_cleaned.csv')
market_data['Date'] = market_data['Date'].astype('datetime64[ns]') 


In [ ]:
AUM.merge(market_data,on="Date").tail(100)

In [ ]:
market_data.loc[df.index[-1] + market_data.offsets.MonthEnd(1)] = df.iloc[-1]
df.resample('d').ffill()